In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from scipy import random as rm, linalg

In 'Binomial Distribution.ipynb', I used sklearn to see if it could estimate our betas.  It performed very poorly.  I wonder if there is a problem with our original data generation, may be they were correlated.  I found this new resource: https://beckernick.github.io/logistic-regression-from-scratch/ and am going to try to see if this works.

The below code only works for two p's.  I'm having trouble with the covariance matrix.

In [90]:
#testing the new branch

In [2]:
def generateNormalVariables(n, normalVar):
    p = normalVar# number of normal predictors
    mu_0_cases = []
    mu_1_cases = []
    cov=[[1, .75] , [.75, 1]] #covariance matrix must be positive semidefinite (a.k.a. nonnegative-definite). Otherwise, the behavior of this method is undefined and backwards compatibility is not guaranteed.
    np.random.seed(12)
    
    #create list of random mus for the 0 labels, list is p long.  
    for index in range (0,p):
        mu_0_cases.append(random.randrange(-5,5))

    #create list of random mus for the 1 labels, list is p long.  
    for index in range (0,p):
        mu_1_cases.append(random.randrange(-5,5))
        
    
    
    #We have to generate two slightly different samples to label zeroes and ones.  These have to be semi-separable....
    x0 = np.random.multivariate_normal(mu_0_cases, cov, num_observations)
    x1 = np.random.multivariate_normal(mu_1_cases, cov, num_observations)
    
    simulated_separableish_features = np.vstack((x0, x1)).astype(np.float32)
    simulated_labels = np.hstack((np.zeros(num_observations), np.ones(num_observations)))

In [92]:
Xnorm = generateNormalVariables(30, 2)
Xnorm = pd.DataFrame.from_dict(Xnorm)

In [117]:
#https://stackoverflow.com/questions/619335/a-simple-algorithm-for-generating-positive-semidefinite-matrices

p = 4
n = 30
A = rm.rand(p,n)
B = np.dot(A,A.transpose())
B=B.diagonal()
print ('random positive semi-define matrix diagonals for today is', B)

random positive semi-define matrix diagonals for today is [ 5.62391206  8.95704146 12.7300451  10.5186036 ]


In [28]:
def generateCountVariables(n, countVar, maxCount):
    X = {}
    
    for i in range(countVar):
        ### In this part we are generating variable, which takes discrete/count values
        ### Earlier we provided arbitary max count (for example: max number of visit to doctor is 6)
        ### Then I generate a population of (n x maxCount) entries. Each entry occurs n times in the pop
        ### Finally after shuffling the population, we take a random sample of n elements, which is X_i
        
        maxCt = maxCount[i]
        counts = np.arange(maxCt)
        population = np.repeat(counts , n)
        random.shuffle(population)
        X[i] = random.sample(list(population), n)
    return X

In [29]:
def generateCategoricalVariables(n, catVar, catNumber):
    temp = {}
    
    for i in range(catVar):
        ### We follow the same principle as count variables. 
        ### But, finally we convert the categorical variables to dummy variables
        ### Note: We use **** k dummy variables **** for k categories
        
        
        noOfCat = catNumber[i]
        categories = np.arange(noOfCat)
        population = np.repeat(categories , n)
        random.shuffle(population)
        temp[i] = random.sample(list(population), n)
    X = pd.DataFrame.from_dict(temp)
    X = pd.get_dummies(X,columns=X.columns,drop_first=False)
    return X

In [30]:
def generateRandomIntercept(n):
    ### For each observation (X) we generate an intercept
    np.random.seed(seed = 123)
    intercept =  np.random.normal(0, 1, n)*5
    return intercept

In [31]:
### The following function also needs some cleaning up. 
### It is just generating all types of variables and putting them together
### We also define how many observations, how many of each variable we need to generate here

def generateX():
    
    n = int(input('Enter number of observations:'))
#     normalVar = int(input('Enter number of variables normally distributed:'))
    normalVar = 5
    Xnorm = generateNormalVariables(n, normalVar)
    Xnorm = pd.DataFrame.from_dict(Xnorm)
    
    
#     catVar = int(input('Enter number of categorical variables:'))
#     catNumber = input('Enter the numbers separated by space: each number correspond to number of categories:')
#     catNumber = list(map(int, catNumber.split()))
    catVar = 1
    catNumber = [2]
    Xcat = generateCategoricalVariables(n, catVar, catNumber)#Make sure catvar = len(catNumber)
    
    
#     countVar = int(input('Enter number of count variables:'))
#     maxCount = input('Enter the numbers separated by space: each number correspond to maxCount:')
#     maxCount = list(map(int, maxCount.split()))
    countVar = 1
    maxCount = [4]
    Xcnt = generateCountVariables(n, countVar, maxCount)#Make sure countVar = len(maxCount)
    Xcnt = pd.DataFrame.from_dict(Xcnt)

    
    X = pd.concat([Xnorm,Xcat,Xcnt],ignore_index=True,axis=1)
    
    intercept = generateRandomIntercept(n)
    intercept = pd.DataFrame(intercept.reshape(n,1))
    X = pd.concat([intercept,X],axis = 1,ignore_index=True)#X with intercept
    return X

In [32]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [33]:
### This function generates response variable (y)
### As input with X matrix n x (p+1), and beta vector (p+1) x 1
### We also provide the name of the distribution as a quoted string

def generateResponseVariable(X, beta, dist):
    if dist == 'bernoulli':
        meanValues = sigmoid(X.dot(beta)) 
    elif dist == 'poisson':
        meanValues = np.exp(X.dot(beta))
    elif dist == 'exponential':
        meanValues = -1/(X.dot(beta))
    else:
        print('please spell check distribution name, all lowercase: bernoulli, poisson or exponential')
        
    
    meanValues = np.array([item for sublist in meanValues.values for item in sublist])
    y = []
    
    for eachMean in meanValues:
        np.random.seed(int(round(eachMean)))
        if dist == 'bernoulli':
            randomPrediction = np.random.binomial(1,eachMean)
        elif dist == 'poisson':
            randomPrediction = np.random.poisson(eachMean)
        elif dist == 'exponential':
            randomPrediction = np.random.exponential(eachMean)
        y.append(randomPrediction)
        print(randomPrediction, eachMean)
    
    return y

In [34]:
### This generates p random beta and 1 as intercept (which we can consider as true beta)
### This function needs work
### Either we have to find beta's, which work for all three distributions
### Or we can generate different sets of beta's for different distribution
### I think, same beta's would be better for comparison


def generateRandomBeta(p):
    beta = {}
    for j in range(p-1):
        random.seed(j)#seed for random number generator
        beta[j] = (j*0.5+1)*(-1)**j#random beta for each predictor
    
    beta = pd.DataFrame(list(beta.items()))
    beta = beta.drop([0],axis=1)
    
    beta.loc[-1] = [1]  
    beta.index = beta.index + 1  
    beta = beta.sort_index()# Beta with intercept
    return beta

In [35]:
### This little function is just commanding to generate X, beta and y
def generateData(dist):
    X = generateX()

    n,p = X.shape
    beta = generateRandomBeta(p)

    y = generateResponseVariable(X, beta, dist) ### dist means pass distribution name as string
    return X, beta, y

In [36]:
#generateData('bernoulli')

In [57]:
#write bernoulli data to .npy for permanent random copy
df=generateData('bernoulli')

Enter number of observations:30


ValueError: mean must be 1 dimensional

In [13]:
np.save('bernoulli_X',df[0])
np.save('bernoulli_betas',df[1])
np.save('bernoulli_y',df[2])

In [14]:
### Look at the poisson responses
### The dispersion is too high :(
### One reason that the X's and beta's need careful consideration
generateData('poisson')

Enter number of observations:2
0 9.283452093879888e-05
7867 7847.5342404768735


(          0         1         2         3         4         5  6  7
 0 -5.428153  0.668536  0.553898  0.840467  2.393134  0.035891  1  1
 1  4.986727  0.151650 -0.393576  1.694675  0.553478  0.244519  1  1,      1
 0  1.0
 1  1.0
 2 -1.5
 3  2.0
 4 -2.5
 5  3.0
 6 -3.5
 7  4.0, [0, 7867])

In [17]:
### All X.beta's have to be negative  :(
### One reason that the generation function of X and beta needs work
generateData('exponential')

Enter number of observations:5
0.04475053649569026 0.05622813149145228


ValueError: scale < 0

# Below are the individual functions, which I accumulated above. If you guys think the individual functions are better.....

In [83]:
# def generateBernoulliResponse(X, beta):
#     meanValues = sigmoid(X.dot(beta)) ### Applying mean function
#     meanValues = np.array([item for sublist in meanValues.values for item in sublist])
#     y = []
#     for eachMean in meanValues:
#         np.random.seed(int(round(eachMean)))
#         randomPrediction = np.random.binomial(1,eachMean) ### predict a random number using the mean
#         y.append(randomPrediction)
#         print(randomPrediction, eachMean)
#     return y

In [88]:
# def generatePoissonResponse(X, beta):
#     meanValues = np.exp(X.dot(beta)) ### Applying mean function
#     meanValues = np.array([item for sublist in meanValues.values for item in sublist])
#     y = []
#     for eachMean in meanValues:
#         np.random.seed(int(round(eachMean)))
#         randomPrediction = np.random.poisson(eachMean) ### predict a random number using the mean
#         y.append(randomPrediction)
#         print(randomPrediction, eachMean)
#     return y

In [ ]:
# def generateExponentialResponse(X, beta):
#     meanValues = -1/X.dot(beta) ### Applying mean function
#     meanValues = np.array([item for sublist in meanValues.values for item in sublist])
#     y = []
#     for eachMean in meanValues:
#         np.random.seed(int(round(eachMean)))
#         randomPrediction = np.random.exponential(eachMean) ### predict a random number using the mean
#         y.append(randomPrediction)
#         print(randomPrediction, eachMean)
#     return y

In [95]:
# def generateBernoulliData():
#     X = generateX()

#     n,p = X.shape
#     beta = generateRandomBeta(p)

#     y = generateBernoulliResponse(X, beta)
#     return X, beta, y

In [96]:
# def generatePoissonData():
#     X = generateX()

#     n,p = X.shape
#     beta = generateRandomBeta(p)

#     y = generatePoissonResponse(X, beta)
#     return X, beta, y

In [97]:
# def generateExponentialData():
#     X = generateX()

#     n,p = X.shape
#     beta = generateRandomBeta(p)

#     y = generateExponentialResponse(X, beta)
#     return X, beta, y